In [9]:
from PIL import Image
import torchvision.transforms as T
import os
import numpy as np

In [2]:
run_id = "1653389330"
root_path = "../data/"
in_path = f"{root_path}{run_id}/"

processed_path = f"{root_path}processed/{run_id}/"
states_path, frames_path = f"{in_path}states", f"{in_path}images"

try:
    os.mkdir(processed_path)
except OSError as error:
    pass

In [16]:
def process_state(did, ids):
    pass


def process_frame(did, ids):
    image = Image.open(os.path.join(frames_path, f"{did}.gif"))
    
    # transform and save frames corrseponding to ids 
    # transform = T.Compose([T.CenterCrop(256), T.Resize(128), T.Grayscale(), T.ToTensor()])
    # images = torch.stack((transform(Image.open('svn_.jpg')), 
    #                       transform(Image.open('svn.jpg'))),  dim=0)
    
    image.close()

In [18]:
for filename in os.listdir(frames_path):
    if filename.endswith(".gif"):
        demonstration_id = int(filename.strip(".gif"))
        
        image = Image.open(os.path.join(frames_path, filename))
        nof_steps = image.n_frames
        image.close()
        
        ids = np.linspace(1, nof_steps, num=400, dtype=int)  # 400 steps on each trajectory
        
        process_state(demonstration_id, ids)
        process_frame(demonstration_id, ids)
        break

../data/1653389330/images/2485.gif
694
../data/1653389330/images/2485.gif
694
